# Deploying AI Models from Multiple Frameworks on same GPU using MME

Amazon SageMaker multi-model endpoints(MME) provide a scalable and cost-effective way to deploy large number of deep learning models. Previously, customers had limited options to deploy 100s of deep learning models that need accelerated compute with GPUs. Now, customers can deploy 1000s of deep learning models on GPUs behind one SageMaker endpoint. MME will run multiple models on a GPU, share GPU instances behind an endpoint across multiple models and dynamically load/unload models based on the incoming traffic. With this, customers can significantly save cost and achieve the best price performance.

In this section we show how MME on GPU allows you to deploy ML models from different frameworks like PyTorch, TensorRT, TensorFlow, ONNX, etc on the same GPU. In this example, we show the deployment of PyTorch and TensorRT DistilBERT models on same GPU using SageMaker MME

<div class="alert alert-info"> 💡 <strong> NOTE </strong>
Set <strong> conda_python3 </strong> as kernel when prompted to set the kernel for this notebook. This notebook was tested with the <strong> conda_python3 </strong> kernel on an Amazon SageMaker notebook instance of type <strong> g5.xlarge </strong>.
</div>

### Installs <a class="anchor" id="installs-and-set-up"></a>

Install required packages

In [6]:
!pip install -qU pip boto3 sagemaker awscli tritonclient[http] transformers datasets

### Imports and variables

In [7]:
# imports
import boto3
import sagemaker
from sagemaker import get_execution_role
import time
import random
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from utils import print_safe

# sagemaker variables
prefix = "nlp-mme-gpu"
role = get_execution_role()
sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client("sagemaker-runtime")
sagemaker_session = sagemaker.Session(boto_session=boto3.Session())
bucket = sagemaker_session.default_bucket()

## Creating Model Artifacts <a class="anchor" id="pytorch-efficientnet-model"></a>


This section presents overview of steps to prepare HuggingFace DistilBERT models to be deployed on SageMaker MME using Triton Inference server model configurations. 

### Prepare PyTorch Model  <a class="anchor" id="create-pytorch-model"></a>


* We load pre-trained PyTorch [Huggingface DistilBERT model](https://huggingface.co/bergum/xtremedistil-emotion) that was finetuned on emotion classification task. 
* Export it to Torchscript serialized format

To perform these steps, we will be using our [pt_exporter.py](./workspace/pt_exporter.py) script and running it within the [PyTorch NGC](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/pytorch) container.

In [8]:
%%writefile ./workspace/pt_exporter.py
import torch
from transformers import AutoModelForSequenceClassification

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForSequenceClassification.from_pretrained("bergum/xtremedistil-emotion", torchscript=True)

model = model.eval()
model = model.to(device)

bs = 224
seq_len = 128
dummy_inputs = [
    torch.randint(1000, (bs, seq_len)).to(device),
    torch.ones(bs, seq_len, dtype=torch.int).to(device),
    torch.zeros(bs, seq_len, dtype=torch.int).to(device),
]

traced_model = torch.jit.trace(model, dummy_inputs)
torch.jit.save(traced_model, "model.pt")

Overwriting ./workspace/pt_exporter.py


Run the cell below to finish preparing the PyTorch DistilBERT model.

In [9]:
!docker run --gpus=all --rm -it --ipc=host --ulimit memlock=-1 --ulimit stack=67108864 \
            -v `pwd`/workspace:/workspace -w /workspace nvcr.io/nvidia/pytorch:22.12-py3 \
            /bin/bash generate_model_pytorch.sh


== PyTorch ==

NVIDIA Release 22.12 (build 49968248)
PyTorch Version 1.14.0a0+410ce96

Container image Copyright (c) 2022, NVIDIA CORPORATION & AFFILIATES. All rights reserved.

Copyright (c) 2014-2022 Facebook Inc.
Copyright (c) 2011-2014 Idiap Research Institute (Ronan Collobert)
Copyright (c) 2012-2014 Deepmind Technologies    (Koray Kavukcuoglu)
Copyright (c) 2011-2012 NEC Laboratories America (Koray Kavukcuoglu)
Copyright (c) 2011-2013 NYU                      (Clement Farabet)
Copyright (c) 2006-2010 NEC Laboratories America (Ronan Collobert, Leon Bottou, Iain Melvin, Jason Weston)
Copyright (c) 2006      Idiap Research Institute (Samy Bengio)
Copyright (c) 2001-2004 Idiap Research Institute (Ronan Collobert, Samy Bengio, Johnny Mariethoz)
Copyright (c) 2015      Google Inc.
Copyright (c) 2015      Yangqing Jia
Copyright (c) 2013-2016 The Caffe contributors
All rights reserved.

Various files include modifications (c) NVIDIA CORPORATION & AFFILIATES.  All rights reserved.

This 

#### Setup PyTorch Model Repository

Now that model artifact is ready we need to set up a model repository containing the model artifact we want to serve along with a [model configuration](https://github.com/triton-inference-server/server/blob/main/docs/model_configuration.md) file i.e. `config.pbtxt`. This is the expected structure of the model repository:
```
xdistilbert_pt
├── 1
│   └── model.pt
└── config.pbtxt
```

#### PyTorch Model configuration <a class="anchor" id="create-pytorch-model-config"></a>

Model configuration file `config.pbtxt` contains the following:  
- `name`: xdistilbert_pt
- `backend`: pytorch
- `max_batch_size`: maximum batch size 224 that the model supports
- `input` and `output` tensor shapes with the `data_type` 

Additionally, you can specify [instance_group](https://github.com/triton-inference-server/server/blob/main/docs/user_guide/model_configuration.md#instance-groups) and [dynamic_batching](https://github.com/triton-inference-server/server/blob/main/docs/user_guide/model_configuration.md#dynamic-batcher) properties to achieve high performance inference. 

In [10]:
!mkdir -p model_repository/xdistilbert_pt/1
!cp workspace/model.pt model_repository/xdistilbert_pt/1/

In [11]:
%%writefile model_repository/xdistilbert_pt/config.pbtxt
name: "xdistilbert_pt"
backend: "pytorch"
max_batch_size: 224
input [
  {
    name: "INPUT__0"
    data_type: TYPE_INT32
    dims: [128]
  },
  {
    name: "INPUT__1"
    data_type: TYPE_INT32
    dims: [128]
  },
    {
    name: "INPUT__2"
    data_type: TYPE_INT32
    dims: [128]
  }
]
output [
  {
    name: "OUTPUT__0"
    data_type: TYPE_FP32
    dims: [6]
  }
]
instance_group {
  count: 1
  kind: KIND_GPU
}

Overwriting model_repository/xdistilbert_pt/config.pbtxt


### Prepare TensorRT Model <a class="anchor" id="create-tensorrt-model"></a>

- We load pre-trained xdistilbert PyTorch model from Huggingface
- Convert to onnx representation using torch onnx exporter.
- Use TensorRT `trtexec` bash command to create the TensorRT model plan

To perform these steps, we will be running the [generate_model_trt.sh](./workspace/generate_model_trt.sh) script inside the [PyTorch NGC](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/pytorch) container.

In [12]:
%%writefile ./workspace/generate_model_trt.sh

echo "Installing Transformers..."
pip -q install transformers[onnx]

echo "Exporting model to ONNX..."
python -m transformers.onnx --model=bergum/xtremedistil-emotion \
                            --feature=sequence-classification /workspace/onnx/

export CUDA_MODULE_LOADING=LAZY
echo "Converting ONNX Model to TensorRT FP16 Plan..."
trtexec --onnx=/workspace/onnx/model.onnx \
        --saveEngine=/workspace/model.plan \
        --minShapes=input_ids:1x128,attention_mask:1x128,token_type_ids:1x128 \
        --optShapes=input_ids:16x128,attention_mask:16x128,token_type_ids:16x128 \
        --maxShapes=input_ids:224x128,attention_mask:224x128,token_type_ids:224x128 \
        --fp16 \
        --verbose \
        --memPoolSize=workspace:14000 | tee conversion_trt.txt

echo "Finished exporting all models..."

Overwriting ./workspace/generate_model_trt.sh


Execute the cell below to finish preparing the TensorRT DistilBERT model.

**Note**:
This TensorRT optimization step takes around 10 minutes to complete. While the step is running, feel free to check out the logs in the cell below and watch this [video](https://www.nvidia.com/en-us/on-demand/session/gtcspring22-s41306/?start=349) to learn more about the different TensorRT optimizations like *Kernel AutoTuning*, *Layer Fusion* and *Reduced Mixed Precision*.

In [13]:
!docker run --gpus=all --rm -it --ipc=host --ulimit memlock=-1 --ulimit stack=67108864 \
            -v `pwd`/workspace:/workspace -w /workspace nvcr.io/nvidia/pytorch:22.12-py3 \
            /bin/bash generate_model_trt.sh


== PyTorch ==

NVIDIA Release 22.12 (build 49968248)
PyTorch Version 1.14.0a0+410ce96

Container image Copyright (c) 2022, NVIDIA CORPORATION & AFFILIATES. All rights reserved.

Copyright (c) 2014-2022 Facebook Inc.
Copyright (c) 2011-2014 Idiap Research Institute (Ronan Collobert)
Copyright (c) 2012-2014 Deepmind Technologies    (Koray Kavukcuoglu)
Copyright (c) 2011-2012 NEC Laboratories America (Koray Kavukcuoglu)
Copyright (c) 2011-2013 NYU                      (Clement Farabet)
Copyright (c) 2006-2010 NEC Laboratories America (Ronan Collobert, Leon Bottou, Iain Melvin, Jason Weston)
Copyright (c) 2006      Idiap Research Institute (Samy Bengio)
Copyright (c) 2001-2004 Idiap Research Institute (Ronan Collobert, Samy Bengio, Johnny Mariethoz)
Copyright (c) 2015      Google Inc.
Copyright (c) 2015      Yangqing Jia
Copyright (c) 2013-2016 The Caffe contributors
All rights reserved.

Various files include modifications (c) NVIDIA CORPORATION & AFFILIATES.  All rights reserved.

This 

[02/02/2023-19:14:16] [I] [TRT] [MemUsageChange] Init CUDA: CPU +10, GPU +0, now: CPU 23, GPU 248 (MiB)
[02/02/2023-19:14:16] [V] [TRT] Trying to load shared library libnvinfer_builder_resource.so.8.5.1
[02/02/2023-19:14:16] [V] [TRT] Loaded shared library libnvinfer_builder_resource.so.8.5.1
[02/02/2023-19:14:19] [I] [TRT] [MemUsageChange] Init builder kernel library: CPU +444, GPU +120, now: CPU 522, GPU 368 (MiB)
[02/02/2023-19:14:19] [V] [TRT] CUDA lazy loading is enabled.
[02/02/2023-19:14:19] [I] Start parsing network model
[02/02/2023-19:14:19] [I] [TRT] ----------------------------------------------------------------
[02/02/2023-19:14:19] [I] [TRT] Input filename:   /workspace/onnx/model.onnx
[02/02/2023-19:14:19] [I] [TRT] ONNX IR version:  0.0.6
[02/02/2023-19:14:19] [I] [TRT] Opset version:    11
[02/02/2023-19:14:19] [I] [TRT] Producer name:    pytorch
[02/02/2023-19:14:19] [I] [TRT] Producer version: 1.14.0
[02/02/2023-19:14:19] [I] [TRT] Domain:           
[02/02/2023-19:

[02/02/2023-19:14:19] [V] [TRT] Registering tensor: /bert/Unsqueeze_output_0 for ONNX tensor: /bert/Unsqueeze_output_0
[02/02/2023-19:14:19] [V] [TRT] /bert/Unsqueeze [Unsqueeze] outputs: [/bert/Unsqueeze_output_0 -> (-1, 1, -1)[INT32]], 
[02/02/2023-19:14:19] [V] [TRT] Parsing node: /bert/Unsqueeze_1 [Unsqueeze]
[02/02/2023-19:14:19] [V] [TRT] Searching for input: /bert/Unsqueeze_output_0
[02/02/2023-19:14:19] [V] [TRT] /bert/Unsqueeze_1 [Unsqueeze] inputs: [/bert/Unsqueeze_output_0 -> (-1, 1, -1)[INT32]], 
[02/02/2023-19:14:19] [V] [TRT] Original shape: (_, 1, _), unsqueezing to: (_, _, _, _)
[02/02/2023-19:14:19] [V] [TRT] Registering layer: /bert/Unsqueeze_1 for ONNX node: /bert/Unsqueeze_1
[02/02/2023-19:14:19] [V] [TRT] Registering tensor: /bert/Unsqueeze_1_output_0 for ONNX tensor: /bert/Unsqueeze_1_output_0
[02/02/2023-19:14:19] [V] [TRT] /bert/Unsqueeze_1 [Unsqueeze] outputs: [/bert/Unsqueeze_1_output_0 -> (-1, 1, 1, -1)[INT32]], 
[02/02/2023-19:14:19] [V] [TRT] Parsing node: 

[02/02/2023-19:14:19] [V] [TRT] Registering tensor: /bert/encoder/layer.0/output/dense/MatMul_output_0 for ONNX tensor: /bert/encoder/layer.0/output/dense/MatMul_output_0
[02/02/2023-19:14:19] [V] [TRT] /bert/encoder/layer.0/output/dense/MatMul [MatMul] outputs: [/bert/encoder/layer.0/output/dense/MatMul_output_0 -> (-1, -1, 256)[FLOAT]], 
[02/02/2023-19:14:19] [V] [TRT] Parsing node: /bert/encoder/layer.0/output/dense/Add [Add]
[02/02/2023-19:14:19] [V] [TRT] Searching for input: bert.encoder.layer.0.output.dense.bias
[02/02/2023-19:14:19] [V] [TRT] Searching for input: /bert/encoder/layer.0/output/dense/MatMul_output_0
[02/02/2023-19:14:19] [V] [TRT] /bert/encoder/layer.0/output/dense/Add [Add] inputs: [bert.encoder.layer.0.output.dense.bias -> (256)[FLOAT]], [/bert/encoder/layer.0/output/dense/MatMul_output_0 -> (-1, -1, 256)[FLOAT]], 
[02/02/2023-19:14:19] [V] [TRT] Registering layer: bert.encoder.layer.0.output.dense.bias for ONNX node: bert.encoder.layer.0.output.dense.bias
[02/0

[02/02/2023-19:14:19] [V] [TRT] Registering layer: /bert/encoder/layer.1/attention/output/dense/Add for ONNX node: /bert/encoder/layer.1/attention/output/dense/Add
[02/02/2023-19:14:19] [V] [TRT] Registering tensor: /bert/encoder/layer.1/attention/output/dense/Add_output_0 for ONNX tensor: /bert/encoder/layer.1/attention/output/dense/Add_output_0
[02/02/2023-19:14:19] [V] [TRT] /bert/encoder/layer.1/attention/output/dense/Add [Add] outputs: [/bert/encoder/layer.1/attention/output/dense/Add_output_0 -> (-1, -1, 256)[FLOAT]], 
[02/02/2023-19:14:19] [V] [TRT] Parsing node: /bert/encoder/layer.1/attention/output/Add [Add]
[02/02/2023-19:14:19] [V] [TRT] Searching for input: /bert/encoder/layer.1/attention/output/dense/Add_output_0
[02/02/2023-19:14:19] [V] [TRT] Searching for input: /bert/encoder/layer.0/output/LayerNorm/Add_1_output_0
[02/02/2023-19:14:19] [V] [TRT] /bert/encoder/layer.1/attention/output/Add [Add] inputs: [/bert/encoder/layer.1/attention/output/dense/Add_output_0 -> (-1, 

...

[02/02/2023-19:14:20] [V] [TRT] Running: ConstShuffleFusion on onnx::MatMul_859
[02/02/2023-19:14:20] [V] [TRT] ConstShuffleFusion: Fusing onnx::MatMul_859 with (Unnamed Layer* 847) [Shuffle]
[02/02/2023-19:14:20] [V] [TRT] Running: ConstShuffleFusion on bert.encoder.layer.5.attention.self.key.bias
[02/02/2023-19:14:20] [V] [TRT] ConstShuffleFusion: Fusing bert.encoder.layer.5.attention.self.key.bias with (Unnamed Layer* 850) [Shuffle]
[02/02/2023-19:14:20] [V] [TRT] Running: ConstShuffleFusion on onnx::MatMul_862
[02/02/2023-19:14:20] [V] [TRT] ConstShuffleFusion: Fusing onnx::MatMul_862 with (Unnamed Layer* 865) [Shuffle]
[02/02/2023-19:14:20] [V] [TRT] Running: ConstShuffleFusion on bert.encoder.layer.5.attention.self.value.bias
[02/02/2023-19:14:20] [V] [TRT] ConstShuffleFusion: Fusing bert.encoder.layer.5.attention.self.value.bias with (Unnamed Layer* 868) [Shuffle]
[02/02/2023-19:14:20] [V] [TRT] Running: ConstShuffleFusion on /bert/encoder/layer.5/attention/self/Constant_12_outp

[02/02/2023-19:14:20] [V] [TRT] After Myelin optimization: 1 layers
[02/02/2023-19:14:20] [V] [TRT] Applying ScaleNodes fusions.
[02/02/2023-19:14:20] [V] [TRT] After scale fusion: 1 layers
[02/02/2023-19:14:20] [V] [TRT] After dupe layer removal: 1 layers
[02/02/2023-19:14:20] [V] [TRT] After final dead-layer removal: 1 layers
[02/02/2023-19:14:20] [V] [TRT] After tensor merging: 1 layers
[02/02/2023-19:14:20] [V] [TRT] After vertical fusions: 1 layers
[02/02/2023-19:14:20] [V] [TRT] After dupe layer removal: 1 layers
[02/02/2023-19:14:20] [V] [TRT] After final dead-layer removal: 1 layers
[02/02/2023-19:14:20] [V] [TRT] After tensor merging: 1 layers
[02/02/2023-19:14:20] [V] [TRT] After slice removal: 1 layers
[02/02/2023-19:14:20] [V] [TRT] After concat removal: 1 layers
[02/02/2023-19:14:20] [V] [TRT] Trying to split Reshape and strided tensor
[02/02/2023-19:14:20] [V] [TRT] Graph construction and optimization completed in 0.200018 seconds.
[02/02/2023-19:14:20] [V] [TRT] Trying t

[02/02/2023-19:14:49] [V] [TRT]  (foreignNode) Set user's cuda kernel library
[02/02/2023-19:14:49] [V] [TRT]  (foreignNode) Pass fuse_conv_padding is currently skipped for dynamic shapes
[02/02/2023-19:14:49] [V] [TRT]  (foreignNode) Pass pad_conv_channel is currently skipped for dynamic shapes
[02/02/2023-19:14:49] [V] [TRT]  (foreignNode) Pass fuse_conv_padding is currently skipped for dynamic shapes
[02/02/2023-19:14:49] [V] [TRT]  (foreignNode) Pass pad_conv_channel is currently skipped for dynamic shapes
[02/02/2023-19:14:50] [V] [TRT] Tactic: 0x0000000000000000 Time: 1.92907
[02/02/2023-19:14:50] [V] [TRT] Fastest Tactic: 0x0000000000000000 Time: 1.92907
[02/02/2023-19:14:50] [V] [TRT] >>>>>>>>>>>>>>> Chose Runner Type: Myelin Tactic: 0x0000000000000000
[02/02/2023-19:14:50] [V] [TRT] *************** Autotuning format combination: Int32(128,1), Int32(128,1), Int32(128,1), Int32() -> Half(6,1) ***************
[02/02/2023-19:14:50] [V] [TRT] --------------- Timing Runner: {Foreign

[02/02/2023-19:24:25] [I] Engine built in 609.625 sec.
[02/02/2023-19:24:26] [I] [TRT] Loaded engine size: 37 MiB
[02/02/2023-19:24:26] [V] [TRT] Deserialization required 12208 microseconds.
[02/02/2023-19:24:26] [I] [TRT] [MemUsageChange] TensorRT-managed allocation in engine deserialization: CPU +0, GPU +28, now: CPU 0, GPU 28 (MiB)
[02/02/2023-19:24:26] [I] Engine deserialized in 0.0125832 sec.
[02/02/2023-19:24:26] [V] [TRT] Total per-runner device persistent memory is 0
[02/02/2023-19:24:26] [V] [TRT] Total per-runner host persistent memory is 256
[02/02/2023-19:24:26] [V] [TRT] Allocated activation device memory of size 191245824
[02/02/2023-19:24:26] [I] [TRT] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +182, now: CPU 0, GPU 210 (MiB)
[02/02/2023-19:24:26] [V] [TRT] CUDA lazy loading is enabled.
[02/02/2023-19:24:26] [I] Setting persistentCacheLimit to 0 bytes.
[02/02/2023-19:24:26] [V] Using enqueueV3.
[02/02/2023-19:24:26] [I] Using 

Finished exporting all models...


#### Setup TensorRT Model Repository

Similar to PyTorch model, this is the expected structure of the TensorRT model repository:
```
xdistilbert_trt
├── 1
│   └── model.plan
└── config.pbtxt
```

#### TensorRT Model configuration <a class="anchor" id="create-pytorch-model-config"></a>

Model configuration file `config.pbtxt` contains the following  
- `name`: xdistilbert_trt
- `backend`: tensorrt
- `max_batch_size`: maximum batch size 224 that the model supports
- `input` and `output` tensor shapes with the `data_types`

Additionally, you can specify [instance_group](https://github.com/triton-inference-server/server/blob/main/docs/user_guide/model_configuration.md#instance-groups) and [dynamic_batching](https://github.com/triton-inference-server/server/blob/main/docs/user_guide/model_configuration.md#dynamic-batcher) properties to achieve high performance inference. 

In [14]:
!mkdir -p model_repository/xdistilbert_trt/1/
!cp workspace/model.plan model_repository/xdistilbert_trt/1/

In [15]:
%%writefile model_repository/xdistilbert_trt/config.pbtxt
name: "xdistilbert_trt"
backend: "tensorrt"
max_batch_size: 224
input [
  {
    name: "input_ids"
    data_type: TYPE_INT32
    dims: [128]
  },
  {
    name: "attention_mask"
    data_type: TYPE_INT32
    dims: [128]
  },
  {
    name: "token_type_ids"
    data_type: TYPE_INT32
    dims: [128]
  }
]
output [
  {
    name: "logits"
    data_type: TYPE_FP32
    dims: [6]
  }
]
instance_group {
  count: 1
  kind: KIND_GPU
}

Overwriting model_repository/xdistilbert_trt/config.pbtxt


## Export model artifacts to S3 <a class="anchor" id="export-to-s3"></a>

Next, we will package our models as `*.tar.gz` files for uploading to S3. 

In [16]:
pytorch_model_file_name = "xdistilbert_pt.tar.gz"
!tar -C model_repository -czf $pytorch_model_file_name xdistilbert_pt
model_uri_pt = sagemaker_session.upload_data(path=pytorch_model_file_name, key_prefix=prefix)
print_safe(f"PyTorch Model S3 location: {model_uri_pt}")

PyTorch Model S3 location: s3://sagemaker-us-west-2-############/nlp-mme-gpu/xdistilbert_pt.tar.gz


In [17]:
tensorrt_model_file_name = "xdistilbert_trt.tar.gz"
!tar -C model_repository -czf $tensorrt_model_file_name xdistilbert_trt
model_uri_trt = sagemaker_session.upload_data(path=tensorrt_model_file_name, key_prefix=prefix)
print_safe(f"TensorRT Model S3 location: {model_uri_trt}")

TensorRT Model S3 location: s3://sagemaker-us-west-2-############/nlp-mme-gpu/xdistilbert_trt.tar.gz


## Setup GPU Multi-Model Endpoint <a class="anchor" id="deploy-models-with-mme"></a>

We will now setup Multi-Model Endpoint on GPU where we can deploy our DistilBERT PyTorch and TensorRT models.

### SageMaker Triton Container Image

First we define the SageMaker Triton container image we will be using that supports deploying multi-model endpoints with GPU.

In [ ]:
# account mapping for SageMaker MME Triton Image
account_id_map = {
    "us-east-1": "785573368785",
    "us-east-2": "007439368137",
    "us-west-1": "710691900526",
    "us-west-2": "301217895009",
    "eu-west-1": "802834080501",
    "eu-west-2": "205493899709",
    "eu-west-3": "254080097072",
    "eu-north-1": "601324751636",
    "eu-south-1": "966458181534",
    "eu-central-1": "746233611703",
    "ap-east-1": "110948597952",
    "ap-south-1": "763008648453",
    "ap-northeast-1": "941853720454",
    "ap-northeast-2": "151534178276",
    "ap-southeast-1": "324986816169",
    "ap-southeast-2": "355873309152",
    "cn-northwest-1": "474822919863",
    "cn-north-1": "472730292857",
    "sa-east-1": "756306329178",
    "ca-central-1": "464438896020",
    "me-south-1": "836785723513",
    "af-south-1": "774647643957",
}

region = boto3.Session().region_name
if region not in account_id_map.keys():
    raise ("UNSUPPORTED REGION")

base = "amazonaws.com.cn" if region.startswith("cn-") else "amazonaws.com"
mme_triton_image_uri = (
    "{account_id}.dkr.ecr.{region}.{base}/sagemaker-tritonserver:22.12-py3".format(
        account_id=account_id_map[region], region=region, base=base
    )
)

### Define the serving container  <a class="anchor" id="define-container-def"></a>

Next, we define the serving container based on the Triton SageMaker image
* In the container definition, define the `ModelDataUrl` to specify the S3 directory that contains all the models that SageMaker multi-model endpoint will use to load  and serve predictions. 
* Set `Mode` to `MultiModel` to indicate SageMaker should create the endpoint with MME specifications.

In [ ]:
model_data_url = f"s3://{bucket}/{prefix}/"

container = {"Image": mme_triton_image_uri, "ModelDataUrl": model_data_url, "Mode": "MultiModel"}

### Create SageMaker model <a class="anchor" id="create-mme-model-obj"></a>

We then create a sagemaker model from the model files we uploaded to s3 in the previous steps. We do this using the SageMaker boto3 client and [create_model](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model) API. We will pass the container definition to the `create model` API along with ModelName and ExecutionRoleArn.


In [ ]:
ts = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

sm_model_name = f"{prefix}-model-{ts}"
create_model_response = sm_client.create_model(
    ModelName=sm_model_name, ExecutionRoleArn=role, PrimaryContainer=container
)

print_safe("Model Arn: " + create_model_response["ModelArn"])

### Define configuration for the Multi-Model <Endpoint class="anchor" id="config-mme"></a>

Next, we create a multi-model endpoint configuration using [create_endpoint_config](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint_config) boto3 API. Specify an accelerated GPU computing instance in InstanceType **(we will use the same instance type that we are using to host our SageMaker Notebook)**. We recommend configuring your endpoints with at least two instances with real-life use-cases. This allows SageMaker to provide a highly available set of predictions across multiple Availability Zones for the models.




In [ ]:
endpoint_config_name = f"{prefix}-epc-{ts}"
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": ml.g5.xlarge,
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": sm_model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print_safe("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

### Create Multi-Model Endpoint  <a class="anchor" id="create-mme"></a>

Using the above endpoint configuration we create a new sagemaker endpoint and wait for the deployment to finish. The status will change to **InService** once the deployment is successful.

In [ ]:
endpoint_name = f"{prefix}-ep-{ts}"
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)

print_safe("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

In [ ]:
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print_safe("\nArn: " + resp["EndpointArn"])
print("Status: " + status)

## Helper functions to prepare Inference Payload

The following helper functions help us transform sample texts into inference payload that can be sent to Triton server on MME which is what is really serving our PyTorch and TensorRT DistilBERT NLP models.

The `tritonclient` package provides utility methods to generate the payload without having to know the details of the specification. We'll use the following methods to convert our inference request into a binary format which provides lower latencies for inference.

**Note:** With the `binary+json` format, we have to specify the length of the request metadata in the header to allow Triton to correctly parse the binary payload. This is done using a custom Content-Type header `application/vnd.sagemaker-triton.binary+json;json-header-size={}`.

In [ ]:
from utils import tokenize_text, logits2prediction, get_random_text
import tritonclient.http as httpclient
import numpy as np

def _get_tokenized_text_binary(text, input_names, output_names):
    inputs = []
    outputs = []
    input_ids, attention_mask, token_type_ids = tokenize_text(tokenizer, text)
    inputs.append(httpclient.InferInput(input_names[0], input_ids.shape, "INT32"))
    inputs.append(httpclient.InferInput(input_names[1], attention_mask.shape, "INT32"))
    inputs.append(httpclient.InferInput(input_names[2], token_type_ids.shape, "INT32"))

    inputs[0].set_data_from_numpy(input_ids.astype(np.int32), binary_data=True)
    inputs[1].set_data_from_numpy(attention_mask.astype(np.int32), binary_data=True)
    inputs[2].set_data_from_numpy(token_type_ids.astype(np.int32), binary_data=True)
    
    outputs.append(httpclient.InferRequestedOutput(output_names[0], binary_data=True))
    request_body, header_length = httpclient.InferenceServerClient.generate_request_body(
        inputs, outputs=outputs
    )
    return request_body, header_length

def get_tokenized_text_binary_pt(text):
    return _get_tokenized_text_binary(text, ["INPUT__0", "INPUT__1", "INPUT__2"], ["OUTPUT__0"])

def get_tokenized_text_binary_trt(text):
    return _get_tokenized_text_binary(text, ["input_ids", "attention_mask", "token_type_ids"], ["logits"])

def read_response(response, output_name):
    # Parse json header size length from the response
    header_length_prefix = "application/vnd.sagemaker-triton.binary+json;json-header-size="
    header_length_str = response['ContentType'][len(header_length_prefix):]
    # Read response body
    result = httpclient.InferenceServerClient.parse_response_body(response['Body'].read(), header_length=int(header_length_str))
    logits = result.as_numpy(output_name)
    prediction = logits2prediction(logits)
    return prediction

## Invoking Models on Multi-Model Endpoint

Once the endpoint is successfully created, we can send inference request to multi-model endpoint using `invoke_enpoint` API. We specify the TargetModel in the invocation call and pass in the payload for each model type. Sample invocation for PyTorch model and TensorRT model is shown below

### Invoke TensorRT Model <a class="anchor" id="invoke-tensorrt-model"></a>

Let's invoke the same model twice.

In [ ]:
sample_text = get_random_text()
request_body, header_length = get_tokenized_text_binary_trt(sample_text)

start_time = time.time()

response = runtime_sm_client.invoke_endpoint(EndpointName=endpoint_name,
                                  ContentType=f"application/vnd.sagemaker-triton.binary+json;json-header-size={header_length}",
                                  Body=request_body,
                                  TargetModel='xdistilbert_trt.tar.gz')
duration = time.time() - start_time

output_name = 'logits'
prediction = read_response(response, output_name)
print(f"text: {sample_text}\n")
print(f"prediction: {prediction}, took {int(duration * 1000)} ms\n")

In [ ]:
sample_text = get_random_text()
request_body, header_length = get_tokenized_text_binary_trt(sample_text)

start_time = time.time()

response = runtime_sm_client.invoke_endpoint(EndpointName=endpoint_name,
                                  ContentType=f"application/vnd.sagemaker-triton.binary+json;json-header-size={header_length}",
                                  Body=request_body,
                                  TargetModel='xdistilbert_trt.tar.gz')
duration = time.time() - start_time

output_name = 'logits'
prediction = read_response(response, output_name)
print(f"text: {sample_text}\n")
print(f"prediction: {prediction}, took {int(duration * 1000)} ms\n")

Notice the higher latencies on the first invocation of any given model. This is due to the time it takes SageMaker to download the model to the Endpoint instance and then load the model into the inference container. Subsequent invocations of the same model take advantage of the model already being loaded into the inference container and so are fast.

### Invoke PyTorch Model <a class="anchor" id="invoke-pytorch-model"></a>

In [ ]:
sample_text = get_random_text()
request_body, header_length = get_tokenized_text_binary_pt(sample_text)

response = runtime_sm_client.invoke_endpoint(EndpointName=endpoint_name,
                                  ContentType=f"application/vnd.sagemaker-triton.binary+json;json-header-size={header_length}",
                                  Body=request_body,
                                  TargetModel='xdistilbert_pt.tar.gz')

output_name = 'OUTPUT__0'
prediction = read_response(response, output_name)
print(f"text: {sample_text}\n")
print(f"prediction: {prediction}")

# Deploying Hundreds of Models to GPUs using Multi-Model Endpoint

Let's say you are trying to deploy 300 customer-specific distilBERT models which are a mixture of frequently and infrequently accessed models which need GPU acceleration for good inference performance. These models might even be coming from different frameworks like PyTorch, TensorFlow, ONNX, TensorRT.

Deploying these 300 models on GPU instances like `g5.xlarge` using dedicated Single-Model Endpoints would take ~300 instances.

By deploying these models behind a Multi-Model endpoint on GPUs you can end up using ~100x less instances. Thus reducing costs by **100x**. 

## Dynamically adding models to an existing Multi-Model Endpoint

It’s easy to deploy a new model to an existing multi-model endpoint. With the endpoint already running, copy a new set of model artifacts to the same S3 location you set up earlier. Client applications are then free to request predictions from that target model, and Amazon SageMaker handles the rest. 

This step below will take around few minutes to complete as we are copying 300 files to S3

In [ ]:
%%capture
num_models = 300
for i in range(num_models):
    customer_model_name = f"xdistilbert_customer{i}.tar.gz"
    model_copy = f"{model_data_url}{customer_model_name}"
    !aws s3 cp $model_data_url$pytorch_model_file_name $model_copy

In [ ]:
!aws s3 ls $model_data_url

### Helper function to invoke models

In [ ]:
def predict_model(text, model_name, show_latency=False):
    print(f"Using model {model_name} to predict")
    
    request_body, header_length = get_tokenized_text_binary_pt(text)
    
    start_time = time.time()
    
    response = runtime_sm_client.invoke_endpoint(EndpointName=endpoint_name,
                                  ContentType='application/vnd.sagemaker-triton.binary+json;json-header-size={}'.format(header_length),
                                  Body=request_body,
                                  TargetModel=model_name)
    
    duration = time.time() - start_time
    
    prediction = read_response(response, output_name="OUTPUT__0")
    
    if show_latency:
        print(f"prediction: {prediction}, took {int(duration * 1000)} ms\n")
    else:
        print(f"prediction: {prediction}\n")

In [ ]:
predict_model(text=get_random_text(), model_name="xdistilbert_customer1.tar.gz")

## Monitoring GPU MME using Amazon CloudWatch metrics
To make price and performance tradeoffs, you will want to test multi-model endpoints with models and representative traffic from your own application. Amazon SageMaker provides metrics in CloudWatch for multi-model endpoints so you can determine the endpoint usage and the cache hit rate and optimize your endpoint.

* **LoadedModelCount**: The number of models loaded in the containers of the multi-model endpoint
* **GPUUtilization**: Precentage of GPU units that are used by the containers on an instance 
* **GPUMemoryUtilization**: Precentage of GPU memory used by the containers on an instance 
* **DiskUtilization**: Precentage of disk space used by the containers on an instance 
and others

SageMaker MME also provides Model loading metrics such as:

* **ModelLoadingWaitTime**: Time interval for model to be downloaded or loaded
* **ModelUnloadingTime**: Time interval to unload model from container
* **ModelDownloadingTime** Time to download the model from S3
* **ModelCacheHit**: Number of invocations to model that are already loaded onto the container

For more details refer [Monitor Amazon SageMaker with Amazon CloudWatch](https://docs.aws.amazon.com/sagemaker/latest/dg/monitoring-cloudwatch.html)

# Dynamic Model Unloading in Multi-Model Endpoint

In this section, we show dynamic loading and unloading of models based on the resource utilization on the GPU based compute instance. When the `GPUMemoryUtilization` is at the maximum and SageMaker cannot load more models to serve the requests, SageMaker unloads previously loaded models to free up GPU memory. 

In this case, 1 `g5.xlarge` instance's NVIDIA A10G GPU (24GB memory) can handle 270 models in memory and beyond that if requests for newer models are made MME starts unloading LRU models to make room for newer models which can be seen in the CloudWatch logs.

In [ ]:
for i in range(300):
    customer_model_name = f"xdistilbert_customer{i}.tar.gz"
    predict_model(text=get_random_text(), model_name=customer_model_name)

# Autoscaling in Multi-Model Endpoint

Amazon SageMaker multi-model endpoints on GPU fully support automatic scaling.

## Set up AutoScaling Policy

In [ ]:
auto_scaling_client = boto3.client('application-autoscaling')

resource_id='endpoint/' + endpoint_name + '/variant/' + 'AllTraffic' 
response = auto_scaling_client.register_scalable_target(
    ServiceNamespace='sagemaker',
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    MinCapacity = 1,
    MaxCapacity = 2
)

response = auto_scaling_client.put_scaling_policy(
    PolicyName='GPUMemUtil-ScalingPolicy',
    ServiceNamespace='sagemaker',
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount', 
    PolicyType='TargetTrackingScaling',
    TargetTrackingScalingPolicyConfiguration={
        'TargetValue': 75, 
        'CustomizedMetricSpecification':
        {
            'MetricName': 'GPUMemoryUtilization',
            'Namespace': '/aws/sagemaker/Endpoints',
            'Dimensions': [
                {'Name': 'EndpointName', 'Value': endpoint_name},
                {'Name': 'VariantName','Value': 'AllTraffic'}
            ],
            'Statistic': 'Average',
            'Unit': 'Percent'
        },
        'ScaleInCooldown': 600,
        'ScaleOutCooldown': 100
    }
)

print("Autoscaling policy for GPU MME endpoint has been set up")

### During Autoscaling the endpoint is still active

In [ ]:
predict_model(text=get_random_text(), model_name="xdistilbert_customer200.tar.gz")

## Invoke All Models Again

In [ ]:
for i in range(1, 300):
    customer_model_name = f"xdistilbert_customer{i}.tar.gz"
    predict_model(sample_text, customer_model_name, show_latency=True)

## Matching your endpoint configuration to your use case

In some cases, you may opt to reduce costs by choosing an instance type that cannot hold all the targeted models in memory at the same time. Amazon SageMaker unloads models dynamically when it runs out of memory to make room for a newly-targeted model. For infrequently requested models, the dynamic load latency may still be acceptable given the resulting lower costs. In cases with more stringent latency needs, you may opt for larger instance types or more instances. Investing time up front to do use-case specific testing and analysis with your multi-model endpoint will help to best optimize cost while meeting the performance needs of your application.

# Clean Up

### Terminate endpoint and clean up artifacts

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=sm_model_name)